# Augment Sentences with flexion and SpaCy

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append("..")

In [3]:
import flexion
import spacy
import itertools

In [4]:
model = spacy.load("de_core_news_lg")

## Toy Example

In [5]:
dat = [
    'Als Ada Lovelace auf einem Ball den Mathematiker Charles Babbage traf, der sie einlud, die von ihm erfundene „Differenzmaschine“ anzusehen, war sie hellauf begeistert.',
    "Die Maschine konnte selbstständig addieren und subtrahieren, doch Ada war klar, dass die Möglichkeiten damit noch lange nicht erschöpft waren.",
    "Sie träumte davon, dass eine solche Maschine eines Tages sogar Musik abspielen könnte, und ersann so die Idee eines modernen Computers. 1845 legte sie den ersten Algorithmus zur maschinellen Berechnung der Bernoulli-Zahlen vor und wird daher von vielen als erste Computerprogrammiererin der Welt gefeiert."
]

In [6]:
# parse all examples
docs = [[{"id": t.i, "form": t.text, "lemma": t.lemma_,
          "upos": t.pos_, "feats": t.morph.to_dict()} for t in model(item)]
        for item in dat if len(item) > 0]

In [7]:
# extract all NOUNs
lemmata = list(itertools.chain(*[[t.get("lemma") for t in doc if t.get("upos") == "VERB"] for doc in docs]))
print(lemmata)

['treffen', 'einladen', 'ansehen', 'begeistern', 'addieren', 'subtrahieren', 'erschöpfen', 'träumen', 'abspielen', 'ersinnen', 'legen', 'feiern']


In [8]:
augmentations = []
for lemma in lemmata:
    for substitute in lemmata:
        if lemma != substitute:
            for doc in docs:
                tmp = flexion.replace(lemma, substitute, doc)
                augmentations.extend(tmp)

In [9]:
list(set(augmentations))[:5]

['Die Maschine konnte selbstständig träumen und subtrahieren, doch Ada war klar, dass die Möglichkeiten damit noch lange nicht erschöpft waren.',
 'Sie legte davon, dass eine solche Maschine eines Tages sogar Musik abspielen könnte, und ersann so die Idee eines modernen Computers. 1845 legte sie den ersten Algorithmus zur maschinellen Berechnung der Bernoulli-Zahlen vor und wird daher von vielen als erste Computerprogrammiererin der Welt gefeiert.',
 'Sie träumte davon, dass eine solche Maschine eines Tages sogar Musik abspielen könnte, und ersann so die Idee eines modernen Computers. 1845 träumte sie den ersten Algorithmus zur maschinellen Berechnung der Bernoulli-Zahlen vor und wird daher von vielen als erste Computerprogrammiererin der Welt gefeiert.',
 'Als Ada Lovelace auf einem Ball den Mathematiker Charles Babbage addierte, der sie einlud, die von ihm erfundene „Differenzmaschine“ anzusehen, war sie hellauf begeistert.',
 'Sie addierte davon, dass eine solche Maschine eines Tage

## VERB

In [10]:
lemma = "laufen"
synonyms = ["gehen", "rennen", "sprinten", "eilen", "hasten", "hechten", "hetzen", "schnellen"]
sentence = "Die Sportlerin lief am Fluß entlang."

# parse with spacy
doc = [{"id": t.i, "form": t.text, "lemma": t.lemma_,
        "upos": t.pos_, "feats": t.morph.to_dict()} 
       for t in model(sentence)]

# create augmentations
augmentations = []
for substitute in synonyms:
    tmp = flexion.replace(lemma, substitute, doc)
    augmentations.extend(tmp)
    
set(augmentations)

{'Die Sportlerin eilte am Fluß entlang.',
 'Die Sportlerin ging am Fluß entlang.',
 'Die Sportlerin hastete am Fluß entlang.',
 'Die Sportlerin hechtete am Fluß entlang.',
 'Die Sportlerin hetzte am Fluß entlang.',
 'Die Sportlerin rannte am Fluß entlang.',
 'Die Sportlerin schnellte am Fluß entlang.',
 'Die Sportlerin sprintete am Fluß entlang.'}

## NOUN

In [11]:
lemma = "Werkstatt"
synonyms = ["Garage", "Gebäude", "Haus", "Eingang"]
sentence = "Sie gehen in die Werkstatt dort, die groß ist."
# sentence = "Leute gehen in die Werkstatt dort, die groß sind."

# parse with spacy
doc = [{"id": t.i, "form": t.text, "lemma": t.lemma_, # "head": t.head.i, 
        "upos": t.pos_, "feats": t.morph.to_dict()} 
       for t in model(sentence)]

# create augmentations
augmentations = []
for substitute in synonyms:
    tmp = flexion.replace(lemma, substitute, doc)
    augmentations.extend(tmp)
    
set(augmentations)

{'Sie gehen in das Gebäude dort, die groß ist.',
 'Sie gehen in das Haus dort, die groß ist.',
 'Sie gehen in den Eingang dort, die groß ist.',
 'Sie gehen in die Garage dort, die groß ist.'}

(Open issue: if the lemma is root of clause sentence (PronType: Rel) the change genus of the relative pronoun)